In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import pickle

In [12]:
def chooseModel(province,inputshape):
    if(province=='BKK'):
        modelLSTM_BKK = Sequential()

        modelLSTM_BKK.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 80, activation = 'relu' ,return_sequences = True))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 80, activation = 'relu' ,return_sequences = True))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_BKK.add(Dropout(0.2))


        modelLSTM_BKK.add(Dense(units = 12))
        return modelLSTM_BKK
    if(province=='Chiangmai'):
        modelLSTM_Chiangmai = Sequential()

        modelLSTM_Chiangmai.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Chiangmai.add(Dropout(0.2))
        # modelLSTM_Chiangmai.add(LSTM(units = 80, activation = 'relu' ,return_sequences = True))
        # modelLSTM_Chiangmai.add(Dropout(0.2))
        modelLSTM_Chiangmai.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_Chiangmai.add(Dropout(0.2))


        modelLSTM_Chiangmai.add(Dense(units = 12))
        return modelLSTM_Chiangmai
    if(province=='Khonkaen'):
        modelLSTM = Sequential()

        modelLSTM.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM.add(Dropout(0.2))

        # modelLSTM.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
        # modelLSTM.add(Dropout(0.2))

        modelLSTM.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM.add(Dropout(0.2))

        modelLSTM.add(Dense(units = 12))
        return modelLSTM
    if(province=='Rayong'):
        modelLSTM_Rayong = Sequential()

        modelLSTM_Rayong.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Rayong.add(Dropout(0.2))

        modelLSTM_Rayong.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_Rayong.add(Dropout(0.2))


        modelLSTM_Rayong.add(Dense(units = 12))
        return modelLSTM_Rayong

    if(province=='Saraburi'):
        modelLSTM_Saraburi = Sequential()

        modelLSTM_Saraburi.add(LSTM(units = 100, activation = 'relu',  input_shape = (inputshape, 4)))
        modelLSTM_Saraburi.add(Dropout(0.2))

        modelLSTM_Saraburi.add(Dense(units = 12))
        return modelLSTM_Saraburi
    if(province=='Surat'):
        modelLSTM_Surat = Sequential()

        modelLSTM_Surat.add(LSTM(units = 80, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Surat.add(Dropout(0.2))
        # modelLSTM_Surat.add(LSTM(units = 120, activation = 'relu', return_sequences = True))
        # modelLSTM_Surat.add(Dropout(0.2))
        modelLSTM_Surat.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_Surat.add(Dropout(0.2))



        modelLSTM_Surat.add(Dense(units = 12))

        return modelLSTM_Surat

In [13]:
all_config ={'BKK':{
                    'batch': 32,
                    'epoch': 10,
},
'Chiangmai':{
                    'batch': 32,
                    'epoch': 10,
},
'Khonkaen':{
                    'batch': 32,
                    'epoch': 10,
}
,'Rayong':{
                    'batch': 16,
                    'epoch': 10,
}
,'Saraburi':{
                    'batch': 16,
                    'epoch': 10,
},'Surat':{
                    'batch': 64,
                    'epoch': 10,
},}

In [14]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']

for province in provinces[1:2]:
    print(province)
    data_training = pd.read_csv('../'+province+'/train/'+province.lower()+'_train_format.csv', date_parser = True)

    data_training = data_training.drop(['date_time', 'lat', 'long'], axis = 1)

    data_test = pd.read_csv(province+'_clean.csv', date_parser = True)

    scaler = MinMaxScaler()
    data_training = scaler.fit_transform(data_training)
    scaler.inverse_transform(data_training)
    scale = 1/scaler.scale_

    X_train = []
    y_train = []

    for i in range(60, data_training.shape[0]):
        X_train.append(data_training[i-60:i])
        a = []
        for j in range(1,13):
            #print(i,j,i-60+j*6)
            a.append(data_training[min(i+j*6,data_training.shape[0]-1),0])
        y_train.append(a)


    X_train, y_train = np.array(X_train), np.array(y_train)



        #create callback
    filepath = province+'_modelLSTM.hdf5'
    checkpoint = ModelCheckpoint(filepath=filepath, 
                                monitor='loss',
                                verbose=0, 
                                save_best_only=True,
                                mode='min')

    earlystopping = EarlyStopping(
        monitor='loss', 
        patience=3, 
        min_delta=0, 
        mode='auto'
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='loss', 
        factor=0.2,   
        patience=2, 
        min_lr=0.001,
        verbose=0
    )
    callbacks = [checkpoint, earlystopping, reduce_lr]

    modelLSTM = chooseModel(province,X_train.shape[1])
    modelLSTM.compile(optimizer='adam', loss = 'mean_squared_error')
    history = modelLSTM.fit(X_train, y_train, epochs=all_config[province]['epoch'], batch_size=all_config[province]['batch'], callbacks=callbacks)

    data_training = pd.read_csv('../'+province+'/train/'+province.lower()+'_train_format.csv', date_parser = True)
    data_training = data_training.drop(['date_time', 'lat', 'long'], axis = 1)
    data_test = data_test.drop(['Unnamed: 0', 'lat', 'long','date_time'], axis = 1)

    past_60_days = data_training.tail(60)
    df = past_60_days.append(data_test, ignore_index = True)
    
    df.fillna(method='ffill', inplace=True)

    inputs = scaler.transform(df)
    
    X_test = []
    y_test = []

    for i in range(60, inputs.shape[0]):
        X_test.append(inputs[i-60:i])
        a = []
        for j in range(1,13):
            #print(i,j,i-60+j*6)
            a.append(inputs[min(i+j*6,inputs.shape[0]-1),0])
        y_test.append(a)
    X_test, y_test = np.array(X_test), np.array(y_test)
    y_pred = modelLSTM.predict(X_test)
    scale = 1/scaler.scale_[0]
    y_pred = y_pred*scale
    
    df1 = pd.DataFrame(y_pred)
    df1.to_csv(province+'_pred.csv',index=False)

    with open('lstm_'+province.lower()+'.pickle', 'wb') as fp:
        pickle.dump(y_pred, fp)

    


Chiangmai
Epoch 1/10
821/821 [==============================] - 65s 77ms/step - loss: 0.0024 - lr: 0.0010
Epoch 2/10
821/821 [==============================] - 74s 91ms/step - loss: 0.0017 - lr: 0.0010
Epoch 3/10
821/821 [==============================] - 76s 93ms/step - loss: 0.0015 - lr: 0.0010
Epoch 4/10
821/821 [==============================] - 79s 96ms/step - loss: 0.0016 - lr: 0.0010
Epoch 5/10
821/821 [==============================] - 78s 95ms/step - loss: 0.0015 - lr: 0.0010
Epoch 6/10
821/821 [==============================] - 78s 95ms/step - loss: 0.0014 - lr: 0.0010
Epoch 7/10
821/821 [==============================] - 77s 94ms/step - loss: 0.0014 - lr: 0.0010
Epoch 8/10
821/821 [==============================] - 78s 95ms/step - loss: 0.0015 - lr: 0.0010
Epoch 9/10
821/821 [==============================] - 76s 92ms/step - loss: 0.0013 - lr: 0.0010
Epoch 10/10
821/821 [==============================] - 76s 93ms/step - loss: 0.0012 - lr: 0.0010


In [15]:
df1.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [16]:
y_pred

array([[15.699754 , 15.049487 , 14.10244  , ..., 14.24078  , 13.507096 ,
        14.209419 ],
       [15.824618 , 15.20435  , 14.210004 , ..., 14.3896265, 13.60528  ,
        14.301791 ],
       [15.952525 , 15.367133 , 14.324774 , ..., 14.532215 , 13.696431 ,
        14.395368 ],
       ...,
       [15.777415 , 15.192277 , 14.058223 , ..., 14.119533 , 13.189267 ,
        14.031493 ],
       [15.836283 , 15.28949  , 14.121713 , ..., 14.210409 , 13.243486 ,
        14.073755 ],
       [15.910774 , 15.398011 , 14.196502 , ..., 14.308846 , 13.303393 ,
        14.130641 ]], dtype=float32)